In [ ]:
# -*- coding: utf-8 -*-
#Lab 7 Amarachi Akunna Onyekachi

import arcpy

class Toolbox(object):
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "My First Toolbox"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [lab7_q1]

#Question 1      
class lab7_q1(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "RMI Tool"
        self.description = "The rank mobility index is a measure of an area's change in population rank among a group of areas"
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(displayName = "Select a feature class: ", name = "inputFC", datatype = "DEFeatureClass", parameterType = "Required", direction = "Input")
        param1 = arcpy.Parameter(displayName = "Select a field to rank: ", name = "field", datatype = "Field", parameterType = "Required", direction = "Input")
        param1.parameterDependencies = [param0.name]
        param2 = arcpy.Parameter(displayName = "Specify Rank field name: ", name = "rankName", datatype = "GPString", parameterType = "Required", direction = "Input")
        params = [param0, param1, param2]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        if not arcpy.ListFields(parameters[0].ValueAsText, parameters[2].ValueAsText):
            arcpy.AddField_management(parameters[0].ValueAsText, parameters[2].ValueAsText, "SHORT")

        recList = []
        recNum = 0

        with arcpy.da.SearchCursor(parameters[0].ValueAsText, [parameters[1].ValueAsText]) as cursor:
            for row in cursor:
                recList.append([recNum, row[0], 0])
                recNum += 1
        sorted_recList = sorted(recList, key = lambda x:x[1], reverse = True)

        rank = 1
        for aRec in sorted_recList:
            aRec[2] = rank
            rank += 1

        reg_recList = sorted(sorted_recList, key = lambda x:x[0])

        with arcpy.da.UpdateCursor(parameters[0].ValueAsText, [parameters[2].ValueAsText]) as rows:
            recNum = 0
            for row in rows:
                row[-1] = reg_recList[recNum][2]
                row[-1] = float(row[-1])
                recNum += 1
                rows.updateRow(row)
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return


    
